### Import Library

In [15]:
import os
import pandas as pd
import numpy as np
from shutil import copyfile

file_root_list = ['noBase', 'noBase_CRD', 'noBase_SfotTriple', 
                  'overBase_Soft_CRD', 'overBase', 'overBase_CRD', 'overBase_SfotTriple', 'overBase_Soft_CRD']


hyper_pd = pd.DataFrame(columns=['File', 'Hyperparameters', 'T-F1', 'S-F1', 'Final'])

for f in file_root_list:
    result = pd.read_csv('./Result/Performance/'+f+'.csv')
    result.columns = ['Hyperparameters', 'T-F1', 'S-F1', 'Final']
    result['File'] = f
    
    hyper_pd = pd.concat([hyper_pd, result])
    
hyper_pd = hyper_pd.sort_values(by=['T-F1', 'S-F1'], ascending=False)
hyper_pd.reset_index(drop=True, inplace=True)

best_hyper_pd = hyper_pd.head().copy()
print('Number of Null: {}'.format(hyper_pd['Final'].isnull().sum()))

Number of Null: 18


In [16]:
best_hyper_pd

,File,Hyperparameters,T-F1,S-F1,Final
0,noBase_CRD,"0.001_1e-08_0.001_[256, 1024, 256]",0.755382,0.506179,0.540033
1,noBase,"0.001_1e-07_0.001_[128, 512, 512, 128]",0.753990,0.508984,0.547629
2,noBase_CRD,"0.001_1e-08_0_[128, 512, 128]",0.752081,0.494190,0.532004
3,noBase,"0.001_1e-07_0.001_[256, 1024, 1024, 256]",0.751294,0.498083,0.535790
4,noBase,"0.001_1e-07_0.001_[256, 1024, 256]",0.751268,0.503652,0.544887


### Train Best Teacher Model

In [7]:
from Model.Base_Model import ANN
from Train.Base_Model import *
from torch.utils.data import DataLoader

In [10]:
# Setting Seed
device = torch.device('cuda:' + str(0)) if torch.cuda.is_available() else torch.device('cpu')
    
CFG = {
        'EPOCHS': 1000,
        'BATCH_SIZE': 256,
        'SEED': 41,
        'In_Hidden': 27,
        'S_In_Hidden': 14,
        'Print_epoch': 50,
        'kd_T': 4,
        'LEARNING_RATE': 0.001,
        'REG': 0.001,
        'In_hidden_list': [256, 1024, 256],
        'MIN_LR': 1e-08,
    }

seed_everything(CFG['SEED'])

# Train
train_dataset = CustomDataset(kind='train', sampling='no')
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)

# Validation
val_dataset = CustomDataset(kind='val')
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [11]:
# Train Teacher Model
model = ANN([CFG['In_Hidden']] + CFG['In_hidden_list'])
model.eval()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'], weight_decay=CFG['REG'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,
                                                               threshold_mode='abs', min_lr=CFG['MIN_LR'],
                                                               verbose=False)
teacher_model, T_F1 = train(model, optimizer, train_loader, val_loader, scheduler, device, CFG)

print('Teacher F1: {}'.format(T_F1))

Epoch [0], Train Loss : [0.31964] Val Loss : [0.28669] Val F1 Score : [0.72201]
Epoch [50], Train Loss : [0.16266] Val Loss : [0.27660] Val F1 Score : [0.73731]
Epoch [100], Train Loss : [0.15063] Val Loss : [0.27918] Val F1 Score : [0.74975]
Epoch [150], Train Loss : [0.15028] Val Loss : [0.27370] Val F1 Score : [0.74861]
Epoch [200], Train Loss : [0.15199] Val Loss : [0.27425] Val F1 Score : [0.74741]
Epoch [250], Train Loss : [0.15137] Val Loss : [0.27764] Val F1 Score : [0.74727]
Epoch [300], Train Loss : [0.15132] Val Loss : [0.27408] Val F1 Score : [0.74629]
Epoch [350], Train Loss : [0.15400] Val Loss : [0.27709] Val F1 Score : [0.74898]
Epoch [400], Train Loss : [0.15024] Val Loss : [0.27585] Val F1 Score : [0.74937]
Epoch [450], Train Loss : [0.15027] Val Loss : [0.27680] Val F1 Score : [0.75127]
Epoch [500], Train Loss : [0.15089] Val Loss : [0.28117] Val F1 Score : [0.74047]
Epoch [550], Train Loss : [0.15026] Val Loss : [0.27520] Val F1 Score : [0.74622]
Epoch [600], Train 

In [14]:
torch.save(teacher_model.state_dict(), './Result/Model/Teacher/teacher.pth')